In [1]:
%%init_spark
launcher.driver_memory = '10G'

# 103 Spark optimizations

The goal of this lab is to understand some of the optimization mechanisms of Spark.

- [Spark programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)
- [RDD APIs](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/RDD.html)
- [PairRDD APIs](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/PairRDDFunctions.html)

In [2]:
import org.apache.spark
import org.apache.spark.HashPartitioner

Intitializing Scala interpreter ...

Spark Web UI available at http://BigData:4040
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1730285310693)
SparkSession available as 'spark'


import org.apache.spark
import org.apache.spark.HashPartitioner


In [3]:
// WEATHER structure: (usaf,wban,year,month,day,airTemperature,airTemperatureQuality)
def parseWeather(row:String) = {
    val usaf = row.substring(4,10)
    val wban = row.substring(10,15)
    val year = row.substring(15,19)
    val month = row.substring(19,21)
    val day = row.substring(21,23)
    val airTemperature = row.substring(87,92)
    val airTemperatureQuality = row.charAt(92)

    (usaf,wban,year,month,day,airTemperature.toInt/10,airTemperatureQuality == '1')
}

// STATION structure: (usaf,wban,city,country,state,latitude,longitude,elevation,date_begin,date_end) 
def parseStation(row:String) = {
    def getDouble(str:String) : Double = {
        if (str.isEmpty)
            return 0
        else
            return str.toDouble
    }
    val columns = row.split(",").map(_.replaceAll("\"",""))
    val latitude = getDouble(columns(6))
    val longitude = getDouble(columns(7))
    val elevation = getDouble(columns(8))
    (columns(0),columns(1),columns(2),columns(3),columns(4),latitude,longitude,elevation,columns(9),columns(10))  
}

parseWeather: (row: String)(String, String, String, String, String, Int, Boolean)
parseStation: (row: String)(String, String, String, String, String, Double, Double, Double, String, String)


In [4]:
val rddWeather = sc
  .textFile("../../../../datasets/big/weather-sample10.txt")
  .map(x => parseWeather(x))
val rddStation = sc
  .textFile("../../../../datasets/weather-stations.csv")
  .map(x => parseStation(x))

rddWeather: org.apache.spark.rdd.RDD[(String, String, String, String, String, Int, Boolean)] = MapPartitionsRDD[2] at map at <console>:30
rddStation: org.apache.spark.rdd.RDD[(String, String, String, String, String, Double, Double, Double, String, String)] = MapPartitionsRDD[5] at map at <console>:33


## 103-1 Simple job optimization

Optimize the two jobs (avg temperature and max temperature) by avoiding the repetition of the same computations and by enforcing a partitioning criteria.
- There are multiple methods to repartition an RDD: check the ```coalesce```, ```partitionBy```, and ```repartition``` methods on the documentation and choose the best one.
  - To create a partitioning function, you must ```import org.apache.spark.HashPartitioner``` and then define ```val p = new HashPartitioner(n)``` where ```n``` is the number of partitions to create
- Verify your persisted data in the web UI
- Verify the execution plan of your RDDs with ```rdd.toDebugString``` (shell only) or on the web UI

In [5]:
// Average temperature for every month
rddWeather
  .filter(_._6<999)
  .map(x => (x._4, x._6))
  .aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1), (a1,a2)=>(a1._1+a2._1,a1._2+a2._2))
  .map({case(k,v)=>(k,Math.round(v._1*100/v._2)/100.0)})
  .collect()

res0: Array[(String, Double)] = Array((10,13.32), (11,8.15), (12,4.08), (01,3.06), (02,5.5), (03,8.31), (04,11.75), (05,15.83), (06,18.53), (07,19.96), (08,20.31), (09,17.24))


In [6]:
// Maximum temperature for every month
rddWeather
  .filter(_._6<999)
  .map(x => (x._4, x._6))
  .reduceByKey((x,y)=>{if(x<y) y else x})
  .collect()

res1: Array[(String, Int)] = Array((10,55), (11,43), (12,47), (01,55), (02,47), (03,44), (04,48), (05,49), (06,56), (07,56), (08,56), (09,55))


In [7]:
val partitionedRddWeather = rddWeather
    .filter(_._6<999)
    .map(x => (x._4, x._6))
    .partitionBy(new HashPartitioner(12))
    .cache()

partitionedRddWeather.count

partitionedRddWeather: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[15] at partitionBy at <console>:29
res2: Long = 4185148


In [8]:
partitionedRddWeather
    .aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1), (a1,a2)=>(a1._1+a2._1,a1._2+a2._2))
    .map({case(k,v)=>(k,Math.round(v._1*100/v._2)/100.0)})
    .collect()

partitionedRddWeather
    .reduceByKey((x,y)=>{if(x<y) y else x})
    .collect()

res3: Array[(String, Int)] = Array((01,55), (02,47), (03,44), (04,48), (05,49), (06,56), (07,56), (10,55), (11,43), (08,56), (09,55), (12,47))


In [9]:
partitionedRddWeather.unpersist(true)

res4: partitionedRddWeather.type = ShuffledRDD[15] at partitionBy at <console>:29


## 103-2 RDD preparation

Check the five possibilities to prepare the Station RDD for subsequent processing and identify the best one.

In [ ]:
// _1 and _2 are the fields composing the key; _4 and _8 are country and elevation, respectively
/*
val rddS1 = rddStation.
  keyBy(x => x._1 + x._2).
  partitionBy(p2).
  cache().
  map({case (k,v) => (k,(v._4,v._8))})
val rddS2 = rddStation.
  keyBy(x => x._1 + x._2).
  map({case (k,v) => (k,(v._4,v._8))}).
  cache().
  partitionBy(p2)
val rddS3 = rddStation.
  keyBy(x => x._1 + x._2).
  partitionBy(p2).
  map({case (k,v) => (k,(v._4,v._8))}).
  cache()
val rddS4 = rddStation.
  keyBy(x => x._1 + x._2).
  map({case (k,v) => (k,(v._4,v._8))}).
  partitionBy(p2).
  cache()
val rddS5 = rddStation.
  map(x => (x._1 + x._2, (x._4,x._8))).
  partitionBy(p2).
  cache()
*/

## 103-3 Joining RDDs

Define the join between rddWeather and rddStation and compute:
- The maximum temperature for every city
- The maximum temperature for every city in the UK: 
  - ```StationData.country == "UK"```
- Sort the results by descending temperature
  - ```map({case(k,v)=>(v,k)})``` to invert key with value and vice versa

Hints & considerations:
- Keep only temperature values <999
- Join syntax: ```rdd1.join(rdd2)```
  - Both RDDs should be structured as key-value RDDs with the same key: usaf + wban
- Consider partitioning and caching to optimize the join
  - Careful: it is not enough for the two RDDs to have the same number of partitions; they must have the same partitioner!
- Verify the execution plan of the join in the web UI

In [10]:
val nPartitions = 12

// STATION structure: (usaf, wban,city,country,state,latitude,longitude,elevation,date_begin,date_end)
val rddS = rddStation
  .map(x => (x._1 + x._2, (x._3, x._4)))
  .partitionBy(new HashPartitioner(nPartitions))
  .setName("RDD_Stations")
  .cache()

// WEATHER structure: (usaf, wban,year,month,day,airTemperature,airTemperatureQuality)
val rddW = rddWeather
  .filter(_._6<999)
  .map(x => (x._1 + x._2, x._6))
  .partitionBy(new HashPartitioner(nPartitions))
  .setName("RDD_Weather")
  .cache()

// JOIN structure: (key, (temperature, city, country))
val rddJ = rddW
  .join(rddS)
  .map({case (key, (temperature, (city, country))) => (key, (temperature, city, country))})
  .partitionBy(new HashPartitioner(nPartitions))

nPartitions: Int = 12
rddS: org.apache.spark.rdd.RDD[(String, (String, String))] = RDD_Stations ShuffledRDD[20] at partitionBy at <console>:32
rddW: org.apache.spark.rdd.RDD[(String, Int)] = RDD_Weather ShuffledRDD[23] at partitionBy at <console>:40
rddJ: org.apache.spark.rdd.RDD[(String, (Int, String, String))] = ShuffledRDD[28] at partitionBy at <console>:48


In [11]:
val rddJMaxTemp = rddJ
    .reduceByKey((x, y) => {if (x._1 > y._1) x else y})
    .partitionBy(new HashPartitioner(nPartitions))
    .setName("RDD_Join_MaxTemp")
    .cache()

rddJMaxTemp: org.apache.spark.rdd.RDD[(String, (Int, String, String))] = RDD_Join_MaxTemp MapPartitionsRDD[29] at reduceByKey at <console>:28


In [12]:
// max temperature per city
val maxTemperaturePerCity = rddJMaxTemp
    .map({case (k, (temp, city, country)) => (city, temp)})

maxTemperaturePerCity.collect

maxTemperaturePerCity: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[30] at map at <console>:28
res5: Array[(String, Int)] = Array((REDWOOD FALLS MUNI AIRPORT,35), (TARO ISLAND,32), (PARNAIBA,32), (NOGLIKI,25), (SALEHARD,27), (ITAITUBA,34), (CUSTER,32), (OUYEN,34), (JINHUA,17), (WENDOVER/AF. AUX. F,37), (MOLOKAI (AMOS),30), (QAISUMAH,47), (NAGASAKI,34), (CEDARA,33), (TRIPOLI INTL,40), (CHIAYI,31), (PRYOR FLD RGNL,36), (BOGUS ROMANIAN,-3), (NEWPORT,20), (ERNEST A. LOVE FIELD ARPT,36), (ASHGABAT,41), (GREENOCK MRCC,24), (TRI CITIES,37), (TANGSHAN,36), (INNSBRUCK,31), (NIAGARA DISTRICT,29), (CENTRE ISLAND,35), (MAASIN,33), (CLAREMORRIS,22), (SAN CLEMENTE ISLAND,26), (MC CHORD FIELD,31), (CAMPO DELL ORO,33), (YARMOUTH,23), (OUARZAZATE,40), (SONNBLICK,9), (SIAULIAI,26), (BODEGA ...


In [13]:
// max temperature per city in UK
val maxTemperaturePerCityInUK = rddJMaxTemp
    .filter({case (key, (temp, city, country)) => country == "UK"})
    .map({case (k, (temp, city, country)) => (city, temp)})

maxTemperaturePerCityInUK.collect

maxTemperaturePerCityInUK: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[32] at map at <console>:29
res6: Array[(String, Int)] = Array((GREENOCK MRCC,24), (SALSBURGH,8), (FOYERS,21), (PLATFORM NO. 62101,21), (BLACKPOOL,25), (FAIRFORD,13), (LARKHILL,27), (PLATFORM NO. 63101,15), (ENVIRONM BUOY 62146,14), (BASTREET,20), (MOEL-Y-CRIO,23), (CHIVENOR,24), (ASPATRIA,25), (SELLA NESS,17), (NEWCASTLE,23), (LEEDS BRADFORD,28), (WICK,18), (ENVIRONM BUOY 62128,17), (MARHAM,31), (FIFE NESS,19), (LOCH GLASCARNOCH,21), (SOUTH THULE IS.,8), (SHOREHAM,27), (SOUTHEND,29), (NEWTON,28), (INVERGORDON HARBOUR,20), (LUTON,28), (DUNDRENNAN,21), (CAIRNWELL,17), (LECONFIELD (AUT),24), (REDESDALE CAMP,23), (SHOBDON AIRFIELD,27), (BRISTOL WEA CENTER,27), (BRIZE NORTON,29), (PRESTWICK,25), (FYLINGDALE...


In [14]:
// results sorting
val sortedMaxTempCity = maxTemperaturePerCity
    .map({case (k, v) => (v, k)})
    .sortByKey(false)
    .collect

val sortedMaxTempCityUK = maxTemperaturePerCityInUK
    .map({case (k, v) => (v, k)})
    .sortByKey(false)
    .collect

sortedMaxTempCity: Array[(Int, String)] = Array((56,SPARREVOHN AFS), (56,INDIAN MOUNTAIN AFS), (56,TIN CITY AFS), (56,GALENA A.), (56,POINT LAY), (56,CAPE LISBURNE AFS), (56,TATALINA LRRS), (55,CAPE NEWENHAM AFS), (55,PUERTO CARRENO), (55,AKJOUJT), (55,BAHAWALPUR), (54,CAPE ROMANZOFF AFS), (53,ALI AL SALEM), (52,""), (51,""), (50,LZ BULL / EXERCISE), (50,SUFFOLK EXECUTIVE), (50,AHWAZ), (50,AGHAJARI), (50,SHAHID ASYAEE), (49,INDIANA CO), (49,KUWAIT INTL), (49,SAFI-ABAD DEZFUL), (49,ABADAN), (48,IMPERIAL CO), (48,K.F.I.A. (KING FAHAD INT. AIRPORT) DAMMA), (48,SIBI), (48,OWATONNA DEGNER RGNL), (48,NEEDLES AIRPORT), (48,AL AHSA), (48,TOUAT CHEIKH SIDI MOHAMED BELKEBIR), (48,TIMIMOUN), (48,NEWCASTLE), (48,ASWAN INTL), (48,DOHA INTL), (48,HASSAKAH), (47,QAISUMAH), (47,KING KHALED MILITARY CIT...


## 103-4 Memory occupation

Use Spark's web UI to verify the space occupied by the provided RDDs.

In [20]:
import org.apache.spark.storage.StorageLevel._

sc.getPersistentRDDs.foreach(_._2.unpersist())

val memRdd = rddWeather.cache()
val memSerRdd = memRdd.map(x=>x).persist(MEMORY_ONLY_SER)
val diskRdd = memRdd.map(x=>x).persist(DISK_ONLY)

import org.apache.spark.storage.StorageLevel._
memRdd: rddWeather.type = MapPartitionsRDD[2] at map at <console>:30
memSerRdd: org.apache.spark.rdd.RDD[(String, String, String, String, String, Int, Boolean)] = MapPartitionsRDD[45] at map at <console>:39
diskRdd: org.apache.spark.rdd.RDD[(String, String, String, String, String, Int, Boolean)] = MapPartitionsRDD[46] at map at <console>:40


In [21]:
memRdd.first
memSerRdd.first
diskRdd.first

res10: (String, String, String, String, String, Int, Boolean) = (010010,99999,2000,01,01,0,true)


In [22]:
memRdd.count
memSerRdd.count
diskRdd.count

res11: Long = 4987830


## 103-5 Evaluating different join methods

Consider the following scenario:
- We have a disposable RDD of Weather data (i.e., it is used only once): ```rddW```
- And we have an RDD of Station data that is used many times: ```rddS```
- Both RDDs are cached (```collect()```is called to enforce caching)

We want to join the two RDDS. Which option is best?
- Simply join the two RDDs
- Enforce on ```rddW1``` the same partitioner of ```rddS``` (and then join)
- Exploit broadcast variables

In [ ]:
import org.apache.spark.HashPartitioner
val p = new HashPartitioner(8)
sc.getPersistentRDDs.foreach(_._2.unpersist())

val rddW = rddWeather.
  filter(_._6<999).
  keyBy(x => x._1 + x._2).
  persist()
val rddS = rddStation.
  keyBy(x => x._1 + x._2).
  partitionBy(p).
  cache()

// Collect to enforce caching
rddW.collect()
rddS.collect()

In [ ]:
// Is it better to simply join the two RDDs..
rddW.
  join(rddS).
  map({case(k,v)=>(v._2._3,v._1._6)}).
  reduceByKey((x,y)=>{if(x<y) y else x}).
  collect

In [ ]:
// ..to enforce on rddW1 the same partitioner of rddS..
rddW.
  partitionBy(p).
  join(rddS).
  map({case(k,v)=>(v._2._3,v._1._6)}).
  reduceByKey((x,y)=>{if(x<y) y else x}).
  collect()

In [ ]:
// ..or to exploit broadcast variables?
val bRddS = sc.broadcast(rddS.map(x => (x._1, x._2._3)).collectAsMap())
val rddJ = rddW.
  map({case (k,v) => (bRddS.value.get(k),v._6)}).
  filter(_._1!=None)
rddJ.
  reduceByKey((x,y)=>{if(x<y) y else x}).
  collect()

## 103-6 Optimizing Exercise 3

Start from the result of the last job of Exercise 3; is there a more efficient way to compute the same result?
- Try it on weather-sample10
- Hint: consider that each station is located in only one country

In [ ]:
import org.apache.spark.HashPartitioner
import org.apache.spark.storage.StorageLevel._
val p = new HashPartitioner(8)
sc.getPersistentRDDs.foreach(_._2.unpersist())

val rddS = rddStation.
  keyBy(x => x._1 + x._2).
  partitionBy(p).
  cache()
val rddW = rddWeather.
  filter(_._6<999).
  keyBy(x => x._1 + x._2).
  partitionBy(p).
  persist(MEMORY_AND_DISK_SER)

// Collect to enforce caching
rddW.collect()
rddS.collect()

In [ ]:
// First version
rddW.
  join(rddS).
  filter(_._2._2._4=="UK").
  map({case(k,v)=>(v._2._3,v._1._6)}).
  reduceByKey((x,y)=>{if(x<y) y else x}).
  map({case(k,v)=>(v,k)}).
  sortByKey(false).
  collect()